# Olympic Medal Prediction - Setup and Database Connection

## 1. Setup and Library Imports

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import warnings
import os
from pathlib import Path
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, precision_recall_curve, accuracy_score, f1_score
)
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

import joblib
from datetime import datetime

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

All libraries imported successfully!
Analysis Date: 2025-07-29 21:24:19


## 2. Database Connection

In [2]:
DB_PATH = '/Users/sophie/Desktop/Uni_year_2/project-at2024-theolympians/Group_Project/olympics_SQL.db'

if not os.path.exists(DB_PATH):
    print(f"Database not found at {DB_PATH}")
    print("Please ensure the database file is in the correct location.")
else:
    print(f"Database found: {DB_PATH}")

    conn = sqlite3.connect(DB_PATH)
    
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(tables_query, conn)
    
    print(f"Available tables: {', '.join(tables['name'].tolist())}")
    print(f"Database size: {os.path.getsize(DB_PATH) / 1024 / 1024:.2f} MB")

Database found: /Users/sophie/Desktop/Uni_year_2/project-at2024-theolympians/Group_Project/olympics_SQL.db
Available tables: COUNTRY, COUNTRY_DETAILS, SPORT, OLYMPIC, SINGLES_EVENT, TEAM_EVENT, OLYMPIC_SPORT, PART_OF, ATHLETE, PARTICIPATES, TEAM
Database size: 6.91 MB


## 3. Data Extraction

In [3]:
data_extraction_query = """
SELECT DISTINCT
    a.athleteID,
    a.name as athlete_name,
    a.height,
    a.weight,
    a.bodyFat,
    a.heartRateVariability,
    a.vo2Max,
    a.bloodOxygen,
    a.injurySeverityScore,
    c.name as country,
    cd.gdp,
    cd.population,
    cd.year as country_year,
    p.ranking,
    p.year as participation_year,
    se.eventName as event_name,
    s.sportName,
    -- Create medal categories
    CASE 
        WHEN p.ranking = 1 THEN 'Gold'
        WHEN p.ranking = 2 THEN 'Silver' 
        WHEN p.ranking = 3 THEN 'Bronze'
        ELSE 'No Medal'
    END as medal_category,
    -- Binary target variable
    CASE 
        WHEN p.ranking <= 3 THEN 1 
        ELSE 0 
    END as has_medal
FROM ATHLETE a
JOIN PARTICIPATES p ON a.athleteID = p.athleteID
JOIN SINGLES_EVENT se ON p.eventID = se.eventID
JOIN SPORT s ON se.sportId = s.sportId
JOIN COUNTRY c ON a.noc = c.noc
JOIN COUNTRY_DETAILS cd ON c.noc = cd.noc AND cd.year = p.year
WHERE p.ranking IS NOT NULL
    AND a.height IS NOT NULL
    AND a.weight IS NOT NULL
    AND cd.gdp IS NOT NULL
    AND cd.population IS NOT NULL
ORDER BY p.year, a.athleteID
"""

try:
    df = pd.read_sql_query(data_extraction_query, conn)
    
    print(f"Successfully extracted {len(df):,} records")
    print(f"Unique athletes: {df['athleteID'].nunique():,}")
    print(f"Countries represented: {df['country'].nunique()}")
    print(f"Sports included: {df['sportName'].nunique()}")
    print(f"Events covered: {df['event_name'].nunique()}")
    
    print(f"\nMedal Distribution:")
    medal_dist = df['medal_category'].value_counts()
    for medal, count in medal_dist.items():
        percentage = (count / len(df)) * 100
        print(f"   {medal}: {count:,} ({percentage:.1f}%)")
    
    print(f"\nSample Data:")
    display(df.head(3))
    
except Exception as e:
    print(f"Error extracting data: {e}")
    print("This might be due to missing tables or different schema structure.")

Successfully extracted 21,398 records
Unique athletes: 12,354
Countries represented: 167
Sports included: 48
Events covered: 266

Medal Distribution:
   No Medal: 19,090 (89.2%)
   Bronze: 907 (4.2%)
   Silver: 708 (3.3%)
   Gold: 693 (3.2%)

Sample Data:


,athleteID,athlete_name,height,weight,bodyFat,heartRateVariability,vo2Max,bloodOxygen,injurySeverityScore,country,gdp,population,country_year,ranking,participation_year,event_name,sportName,medal_category,has_medal
0,7,Patrick Chila,180.0,73.0,9.478610,90.131814,64.186082,99.241007,5.0,France,2.926803e+12,64379696,2008,49,2008,"Singles, Men",Table Tennis,No Medal,0
1,15,Damien Éloi,165.0,58.0,14.560510,67.722398,61.741788,96.418747,3.0,France,2.926803e+12,64379696,2008,33,2008,"Singles, Men",Table Tennis,No Medal,0
2,27,Christophe Legoût,177.0,75.0,9.714766,94.175905,60.406506,98.754808,0.0,France,2.926803e+12,64379696,2008,49,2008,"Singles, Men",Table Tennis,No Medal,0


## 4. Save Data for Next Notebook

In [4]:
if 'df' in locals() and df is not None:
    df.to_pickle('olympic_data_raw.pkl')
    print("Data saved to 'olympic_data_raw.pkl' for use in next notebook.")
    print(f"Final dataset shape: {df.shape}")
else:
    print("No data to save - check database connection and query.")

if 'conn' in locals():
    conn.close()
    print("Database connection closed.")

Data saved to 'olympic_data_raw.pkl' for use in next notebook.
Final dataset shape: (21398, 19)
Database connection closed.
